In [74]:
# Add code for Q15 model
# Improve saving data
# Adjust hyperparameters
# Create figure (in R)
# Test with test data

In [75]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision.transforms import *
# from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

import numpy as np
import math
import os
import glob
from PIL import Image


In [76]:
BATCH_SIZE = 32
EPOCHS = 3
LEARNING_RATE = 0.0013
BATCH_INTERVAL = 100
TRAIN_SIZE = 50000
VALIDATION_SIZE = 10000

In [77]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

def get_dataloader(train_set, BATCH_SIZE):
    return torch.utils.data.DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True, num_workers = 2)

# Load data

In [78]:
train_32 = torchvision.datasets.ImageFolder(root = '../data/mnist-varres/32/train',
                                              transform=transform)
train_48 = torchvision.datasets.ImageFolder(root = '../data/mnist-varres/48/train',
                                              transform=transform)
train_64 = torchvision.datasets.ImageFolder(root = '../data/mnist-varres/64/train',
                                              transform=transform)

In [79]:
train_32, validation_32 = torch.utils.data.random_split(train_32, [math.floor(TRAIN_SIZE/(TRAIN_SIZE+VALIDATION_SIZE)*len(train_32)),
                                                                   math.ceil((1-TRAIN_SIZE/(TRAIN_SIZE+VALIDATION_SIZE))*len(train_32))])
train_48, validation_48 = torch.utils.data.random_split(train_48, [math.floor(TRAIN_SIZE/(TRAIN_SIZE+VALIDATION_SIZE)*len(train_48)),
                                                                    math.ceil((1-TRAIN_SIZE/(TRAIN_SIZE+VALIDATION_SIZE))*len(train_48))])
train_64, validation_64 = torch.utils.data.random_split(train_64, [math.floor(TRAIN_SIZE/(TRAIN_SIZE+VALIDATION_SIZE)*len(train_64)),
                                                                   math.ceil((1-TRAIN_SIZE/(TRAIN_SIZE+VALIDATION_SIZE))*len(train_64))])

In [80]:
train_loader_32 = get_dataloader(train_32, BATCH_SIZE)
train_loader_48 = get_dataloader(train_48, BATCH_SIZE)
train_loader_64 = get_dataloader(train_64, BATCH_SIZE)

In [81]:
validation_loader_32 = get_dataloader(validation_32, BATCH_SIZE)
validation_loader_48 = get_dataloader(validation_48, BATCH_SIZE)
validation_loader_64 = get_dataloader(validation_64, BATCH_SIZE)

# Network architecture

In [82]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        N = 81
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding = 1)  # size: batch, 1, 32, 32
        self.conv2 = nn.Conv2d(16, 32, kernel_size = 3, stride = 1, padding = 1) # (batch, 16, 16, 16)
        self.conv3 = nn.Conv2d(32, N, kernel_size = 3, stride = 1, padding = 1) # (batch, 32, 8, 8)
        self.fc1 = nn.Linear(N, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2) # (batch, 16, 28, 28)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # (batch, 32, 14, 14)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2) # (batch, N, 8, 8)
        x = F.max_pool2d(x, kernel_size = x.size()[2:]) # (batch, N, 4, 4)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim = 1)

In [83]:
network = Net()
optimizer = optim.Adam(network.parameters(), lr=LEARNING_RATE)

In [84]:
train_losses = []
train_counter = []
val_losses = []
val_counter = [i*TRAIN_SIZE for i in range(EPOCHS + 1)]
val_acc = []

In [85]:
def train(epoch, train_loader):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        
        output = network(data)
        
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % BATCH_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append(
                (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
            torch.save(network.state_dict(), '../results/model_Q17variable.pth')
            torch.save(optimizer.state_dict(), '../results/optimizer_Q17variable.pth')

In [86]:
def test(validation_loader, VALIDATION_SIZE):
    network.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            output = network(data)
            val_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        val_loss /= VALIDATION_SIZE
        val_losses.append(val_loss)
        val_acc.append(float(100. * correct / VALIDATION_SIZE))
        print('\nValidation set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            val_loss, correct, VALIDATION_SIZE,
            100. * correct / VALIDATION_SIZE))

# Training and validation

In [87]:
train_loaders = {'32': train_loader_32,
                 '48': train_loader_48,
                 '64': train_loader_64
                }
validation_loaders = {'32': validation_loader_32,
                      '48': validation_loader_48,
                      '64': validation_loader_64
                     }

validation_size = {'32': len(validation_32),
                   '48': len(validation_48),
                   '64': len(validation_64)}

for epoch in range(1, EPOCHS + 1):
    print(f'EPOCH: {epoch}')
    for r in train_loaders.keys():
        print(f'\tRESOLUTION: {r}')
        train(epoch, train_loaders[r])
        test(validation_loaders[r], validation_size[r])

EPOCH: 1
	RESOLUTION: 32
Train Epoch: 1 [0/16594 (0%)]	Loss: 2.321974
Train Epoch: 1 [3200/16594 (19%)]	Loss: 1.874787
Train Epoch: 1 [6400/16594 (39%)]	Loss: 1.072306
Train Epoch: 1 [9600/16594 (58%)]	Loss: 0.380115
Train Epoch: 1 [12800/16594 (77%)]	Loss: 0.560354
Train Epoch: 1 [16000/16594 (96%)]	Loss: 0.533825

Validation set: Avg. loss: 0.5324, Accuracy: 2778/3319 (84%)

	RESOLUTION: 48
Train Epoch: 1 [0/16733 (0%)]	Loss: 0.859010
Train Epoch: 1 [3200/16733 (19%)]	Loss: 0.534421
Train Epoch: 1 [6400/16733 (38%)]	Loss: 0.552761
Train Epoch: 1 [9600/16733 (57%)]	Loss: 0.321906
Train Epoch: 1 [12800/16733 (76%)]	Loss: 0.320573
Train Epoch: 1 [16000/16733 (96%)]	Loss: 0.392217

Validation set: Avg. loss: 0.3465, Accuracy: 2985/3347 (89%)

	RESOLUTION: 64
Train Epoch: 1 [0/16672 (0%)]	Loss: 0.594869
Train Epoch: 1 [3200/16672 (19%)]	Loss: 0.343624
Train Epoch: 1 [6400/16672 (38%)]	Loss: 0.201801
Train Epoch: 1 [9600/16672 (58%)]	Loss: 0.183525
Train Epoch: 1 [12800/16672 (77%)]	Loss: 

# Save data

In [88]:
# Store values
np.savetxt("../results/training_stats_Q17variable.csv", [p for p in zip(train_counter, train_losses)], delimiter=',', fmt='%s')
np.savetxt("../results/validation_stats_Q17variable.csv", [p for p in zip(val_losses, val_acc)], delimiter=',', fmt='%s')

# Measure performance on test set

The following three blocks should only be run during the first time of running this notebook.

In [89]:
# filter the test images according to their resolutions and re-store them 
# RESOLUTIONS = [32,48,64]
# TARGETS = [0,1,2,3,4,5,6,7,8,9]
# ROOT = '../data' # .. to not have data folder within notebooks folder

# for r in RESOLUTIONS:
#     for t in TARGETS:
#         os.makedirs(f'{ROOT}/{r}/test/{t}')
        

In [90]:
##for mac and linux
# for file in list(glob.glob('../data/mnist-varres/test/*/*.png')):
#     target = str.split(file, "/")[-2]
#     filename = str.split(file, "/")[-1]
#     im = Image.open(file)
#     if im.size == (32, 32):
#         im.save(f'{ROOT}/32/test/{target}/{filename}')
#     elif im.size == (48, 48):
#         im.save(f'{ROOT}/48/test/{target}/{filename}')
#     elif im.size == (64,64):
#         im.save(f'{ROOT}/64/test/{target}/{filename}')

FileNotFoundError: [Errno 2] No such file or directory: '../data/64/test/mnist-varres/test\\0\\000000.png'

In [91]:
# # For windows:
# for file in list(glob.glob('../data/mnist-varres/test/*/*.png')):
#     target = str.split(file, "\\")[-2]
#     filename = str.split(file, "\\")[-1]
#     im = Image.open(file)
#     if im.size == (32, 32):
#         im.save(f'{ROOT}/32/test/{target}/{filename}')
#     elif im.size == (48, 48):
#         im.save(f'{ROOT}/48/test/{target}/{filename}')
#     elif im.size == (64,64):
#         im.save(f'{ROOT}/64/test/{target}/{filename}')

In [92]:
def get_dataloader_test(test_set, BATCH_SIZE):
    return torch.utils.data.DataLoader(test_set, batch_size = BATCH_SIZE, shuffle = True, num_workers = 2)

In [93]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

In [94]:
test_32 = torchvision.datasets.ImageFolder(root = '../data/mnist-varres/32/test',
                                              transform=transform)
test_48 = torchvision.datasets.ImageFolder(root = '../data/mnist-varres/48/test',
                                              transform=transform)
test_64 = torchvision.datasets.ImageFolder(root = '../data/mnist-varres/64/test',
                                              transform=transform)

test_loader_32 = get_dataloader(test_32, BATCH_SIZE)
test_loader_48 = get_dataloader(test_48, BATCH_SIZE)
test_loader_64 = get_dataloader(test_64, BATCH_SIZE)

test_loaders = {'32': test_loader_32,
                      '48': test_loader_48,
                      '64': test_loader_64
                     }

test_size = {'32': len(test_32),
                   '48': len(test_48),
                   '64': len(test_64)}



In [95]:
test_losses = [] 
test_acc = []

In [96]:
def test_set_performance(test_loader, TEST_SIZE):

    network.eval()
    test_loss = 0
    
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        test_loss /= TEST_SIZE
        test_losses.append(test_loss)
        test_acc.append(float(100. * correct / TEST_SIZE))
        print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, TEST_SIZE,
            100. * correct / TEST_SIZE))


In [97]:
for r in test_loaders.keys():
    print(f'\tRESOLUTION: {r}')
    test_set_performance(test_loaders[r], test_size[r])
    

	RESOLUTION: 32

Test set: Avg. loss: 0.1568, Accuracy: 3107/3269 (95%)

	RESOLUTION: 48

Test set: Avg. loss: 0.1499, Accuracy: 3225/3381 (95%)

	RESOLUTION: 64

Test set: Avg. loss: 0.1512, Accuracy: 3184/3350 (95%)



In [100]:
# calculate a weighted average of accuracy and avgloss
final_acc = 0
final_avgloss = 0

for i, r in enumerate(test_loaders.keys()): 
    final_acc += test_acc[i] * test_size[r]
    final_avgloss += test_losses[i] * test_size[r]
final_acc /= sum([test_size[r] for r in test_loaders.keys()])
final_avgloss /= sum([test_size[r] for r in test_loaders.keys()])

print(test_size)
print('Final Test Accuracy = ', final_acc, '%')
print('Final Avg. Loss = ', final_avgloss)

{'32': 3269, '48': 3381, '64': 3350}
Final Test Accuracy =  95.16000006637573 %
Final Avg. Loss =  0.15257839945480228


In [99]:
#Store values
np.savetxt("../results/test_stats_Q17variable.csv", [p for p in zip(test_losses, test_acc)], delimiter=',', fmt='%s')